In [8]:
import polars as pl
import pandas as pd

### 1. 读取数据

In [9]:
data_path = "/data3/zxh/news_rec/raw_data"

In [ ]:
# 会发现数据中duration存在-1和619315213这个异常值
train_data = pl.read_ipc(f"{data_path}/train_data.arrow")
train_data.describe()

In [ ]:
user_info = pl.read_ipc(f"{data_path}/user_info.arrow")
user_info

### 2 整理数据

#### 2.1 数据去重

In [30]:
# 总共189_766_959条数据，查看是否存在数据重复的情况
train_data.n_unique()

189766756

In [31]:
# 检查是否存在两条数据消费时长相同的情况，否则直接去重会导致消费时长不准确
duplicate_rows = train_data.filter(
    pl.struct([
        "user_id", "article_id", "expose_time", "network_env", 
        "refresh_count", "expose_pos", "is_clicked", "duration"
    ]).is_duplicated()
)
duplicate_rows.filter(pl.col("is_clicked") == 1)

user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64


In [32]:
# 可以发现不存在这样的情况，全部重复的都是未点击的数据，可以放心去重
train_data = train_data.unique()
train_data

user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64
1502953860,462807710,1624713896530,5,4,17,1,59
1518941836,462480108,1624574038941,2,5,64,1,599
1351060716,465142813,1625187191601,2,2,29,1,161
1653561010,465959015,1625449784058,5,2,22,0,0
2220799818,465227930,1625293593524,5,27,271,0,0
…,…,…,…,…,…,…,…
1453195762,463582153,1624730246407,5,2,25,1,91
2216899802,464156892,1624956251496,5,8,9,0,0
1388193826,463864732,1624846543877,5,2,25,0,0


#### 2.2 处理异常值

In [33]:
# 将is_clicked为0，但是duration为-1的数据置为0，剩下duration为-1的数据都是交互过的数据，先置null，后面再用均值慢慢填
train_data = train_data.with_columns(
    pl.when((pl.col("is_clicked") == 0) & (pl.col("duration") == -1))
    .then(0)
    .otherwise(pl.col("duration"))
    .alias("duration")
).with_columns(
    pl.when(pl.col("duration") == -1)
    .then(None)  # 把 -1 变成 NULL
    .otherwise(pl.col("duration"))
    .alias("duration")
)
train_data.describe()

statistic,user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1.89766756e8,1.89766756e8,1.89766756e8,1.89766756e8,1.89766756e8,1.89766756e8,1.89766756e8,1.8967426e8
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,92496.0
"""mean""",1.8993e9,4.6449e8,1.6251e12,3.080074,8.920252,52.756055,0.142765,24.60584
"""std""",5.5349e8,1.2495e6,2.9934e8,1.437007,15.020521,63.843679,0.349833,44968.480428
"""min""",17340.0,3.25279629e8,1.6245e12,2.0,0.0,0.0,0.0,0.0
"""25%""",1.4664e9,4.63502507e8,1.6248e12,2.0,2.0,17.0,0.0,0.0
"""50%""",2.2081e9,4.64511782e8,1.6251e12,2.0,4.0,32.0,0.0,0.0
"""75%""",2.3976e9,4.65520008e8,1.6253e12,5.0,10.0,64.0,0.0,0.0
"""max""",2.4473e9,4.66910675e8,1.6256e12,5.0,650.0,2698.0,1.0,6.19315213e8


In [34]:
# 将 null 的行和非 null 的行拆分
null_duration_df = train_data.filter(pl.col("duration").is_null())
non_null_duration_df = train_data.filter(pl.col("duration").is_not_null())

#### 2.3 处理重复消费的数据

In [35]:
#（存在一次曝光，重复消费的情况，可以认为duation是这些时长的加和，null已经被排除在外，同时619315213异常值也没重复，所以可以放心地加和）
non_null_duration_df = non_null_duration_df.group_by([
    "user_id", "article_id", "expose_time", "network_env", 
    "refresh_count", "expose_pos", "is_clicked"
]).agg(
    pl.when(pl.col("is_clicked") == 1)  # 只计算有效时长
    .then(pl.col("duration"))
    .otherwise(0)  # 其他情况，duration 视为 0
    .sum()
    .alias("duration")
)
non_null_duration_df.describe()

statistic,user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1.8925703e8,1.8925703e8,1.8925703e8,1.8925703e8,1.8925703e8,1.8925703e8,1.8925703e8,1.8925703e8
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",1.8993e9,4.6449e8,1.6251e12,3.079896,8.930465,52.803684,0.140456,24.660085
"""std""",5.5346e8,1.2496e6,2.9933e8,1.436956,15.027445,63.836005,0.347459,45018.023684
"""min""",17340.0,3.25279629e8,1.6245e12,2.0,0.0,0.0,0.0,0.0
"""25%""",1.4664e9,4.63502507e8,1.6248e12,2.0,2.0,17.0,0.0,0.0
"""50%""",2.2081e9,4.64511782e8,1.6251e12,2.0,4.0,32.0,0.0,0.0
"""75%""",2.3976e9,4.65520008e8,1.6253e12,5.0,11.0,64.0,0.0,0.0
"""max""",2.4473e9,4.66910675e8,1.6256e12,5.0,650.0,2698.0,1.0,6.19315213e8


In [36]:
# 把聚合过程中丢失的NULL行拼接回去
train_data = pl.concat([non_null_duration_df, null_duration_df], how="vertical")
train_data

user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64
2423808534,466366964,1625540396046,2,15,34,0,0
1488774712,464474656,1625067658260,2,6,65,0,0
2227847624,465832824,1625401643135,2,0,14,1,122
1357077312,462976168,1624661743375,2,1,14,0,0
1300963958,465515438,1625471821231,2,6,19,0,0
…,…,…,…,…,…,…,…
2228581228,463527762,1624754405902,2,3,18,1,null
1375903970,465700145,1625384846349,5,28,285,1,null
253737602,465852130,1625402521939,2,32,327,1,null


#### 2.4 收起来

In [12]:
repeat_keys = train_data.group_by([
    "user_id", "article_id", "expose_time", "network_env", 
    "refresh_count", "expose_pos", "is_clicked"
]).agg(pl.count().alias("count")).filter(pl.col("count") >= 3)

# 关联原数据，提取所有满足条件的完整行
repeat_data = train_data.join(
    repeat_keys.drop("count"),  # 只保留分组字段用于匹配
    on=["user_id", "article_id", "expose_time", "network_env", "refresh_count", "expose_pos", "is_clicked"],
    how="inner"
)

repeat_data

<ipython-input-12-25d24651e81b>:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  ]).agg(pl.count().alias("count")).filter(pl.col("count") >= 3)


user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64


In [13]:
train_data.filter((pl.col("user_id") == 1435686090) & (pl.col("article_id") == 465325845))

user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64
1435686090,465325845,1625255569591,2,5,61,1,683
1435686090,465325845,1625255569591,2,5,61,1,null


In [14]:
#（存在一次曝光，重复消费的情况，可以认为duation是这些时长的加和，先把-1的排除在外，同时619315213异常值也没重复，所以可以放心地加和）
train_data = train_data.filter(
    (pl.col("is_clicked") == 1) & (pl.col("duration") != -1)  # 只保留有效点击且 duration ≠ -1
).group_by([
    "user_id", "article_id", "expose_time", "network_env", 
    "refresh_count", "expose_pos", "is_clicked"
]).agg(
    pl.col("duration").sum().alias("duration")  # 计算同一曝光下的点击时长总和
)

train_data

user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64
2437270654,465092006,1625145171010,2,2,17,1,62
2367063372,464467157,1624961787565,2,2,17,1,205
2157786858,465370105,1625435039688,2,3,42,1,43
2230286344,465448028,1625316042749,5,2,27,1,50
118443836,463635906,1624764320571,5,1,7,1,22
…,…,…,…,…,…,…,…
2446032746,465839625,1625400735640,2,3,30,1,56
2433836744,464382231,1625104750958,2,7,77,1,33
2214097968,463528539,1624879527628,2,11,44,1,58


In [15]:
neg_duration_df = train_data.filter(pl.col("duration") == -1)
neg_duration_df

user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64


In [16]:
neg_duration_keys = neg_duration_df.select([
    "user_id", "article_id", "expose_time", "network_env", "refresh_count", "expose_pos"
])
neg_duration_keys

user_id,article_id,expose_time,network_env,refresh_count,expose_pos
i64,i64,i64,i64,i64,i64


In [17]:
matched_rows = train_data.join(
    neg_duration_keys,  # 只包含有 -1 duration 的 key
    on=["user_id", "article_id", "expose_time", "network_env", "refresh_count", "expose_pos"],
    how="inner"
)

# 过滤掉 duration 为 -1 的行
filtered_rows = matched_rows.filter((pl.col("duration") != -1) & (pl.col("is_clicked") == 0))

In [18]:
train_data.filter((pl.col("user_id") == 1869728386) & (pl.col("article_id") == 465285666))

user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64
1869728386,465285666,1625254758845,2,0,12,1,535


In [19]:
# 进行自连接，匹配相同的 user_id, article_id, expose_time, ... 但 duration 不同
conflicting_duration_df = negative_duration_df.join(
    train_data,
    on=["user_id", "article_id", "expose_time", "network_env", 
        "refresh_count", "expose_pos", "is_clicked"],
    how="inner"
).filter(pl.col("duration_right") != -1)  # 确保右侧数据的 duration 不等于 -1
conflicting_duration_df

NameError: name 'negative_duration_df' is not defined

In [ ]:
ttt = train_data.filter(
    (pl.col("is_clicked") == 1) & 
    (pl.struct([
        "user_id", "article_id", "expose_time",
        "network_env", "refresh_count", "expose_pos"
    ]).is_duplicated())  # 在全体数据中计算重复性
).sort(["user_id", "article_id", "expose_time"])

In [ ]:
train_data.filter(pl.col("duration") == -1)

user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64
2415272604,466326803,1625470366122,5,37,170,1,-1
2215265208,463348604,1624787418143,5,32,336,1,-1
2207296234,464669571,1625063491496,5,110,44,1,-1
2443817782,465896280,1625405686434,2,8,82,0,-1
2446226728,463101646,1624803368886,2,32,127,0,-1
…,…,…,…,…,…,…,…
2208465704,463862574,1624854289496,5,14,62,1,-1
1301437954,464294504,1624974732168,5,1,18,1,-1
2432585646,462713312,1624697961185,5,10,79,0,-1


In [ ]:
ttt.filter(pl.col("duration")==-1)

user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64
3306286,463086468,1624603635336,5,21,44,1,-1
4830994,466141343,1625464526405,5,0,12,1,-1
5655650,465724979,1625375012141,5,15,21,1,-1
6287150,463514607,1624808986457,5,24,245,1,-1
6529470,464121124,1624891748632,2,1,17,1,-1
…,…,…,…,…,…,…,…
2447092826,465581023,1625461627033,2,2,21,1,-1
2447106922,465750594,1625482617760,2,1,18,1,-1
2447167992,465699712,1625522649839,2,3,17,1,-1


In [ ]:
train_data.filter((pl.col("user_id") == 3306286) & (pl.col("article_id") == 463086468))

user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64
3306286,463086468,1624603635336,5,21,44,1,-1
3306286,463086468,1624603635336,5,21,44,1,28


user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64
2439555116,464821137,1625247462452,4,33,21,0,0
2392412230,463566140,1624895869555,5,5,37,0,0
1520915528,465822098,1625369709129,5,1,19,1,157
1782103134,464333350,1624941626215,2,3,42,0,0
2434613966,464102926,1624895485965,2,18,187,0,0
…,…,…,…,…,…,…,…
2215126746,464244896,1624935423075,5,9,90,0,0
2389482026,466209894,1625582813474,2,13,135,0,0
1638536864,464718109,1625232679893,5,1,17,1,285


In [ ]:
# 存在一条记录又被记录为点击又被记录为未点击的情况
duplicate_rows = train_data.filter(
    pl.struct([
        "user_id", "article_id", "expose_time",
        "network_env", "refresh_count", "expose_pos"
    ]).is_duplicated()
).sort(["user_id","article_id","expose_time"])
duplicate_rows

user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64
17340,462494255,1624664193482,5,4,50,0,0
17340,462494255,1624664193482,5,4,50,1,125
17340,462545264,1624682740741,5,9,17,1,113
17340,462545264,1624682740741,5,9,17,0,0
17340,462792370,1624677924139,5,2,34,0,0
…,…,…,…,…,…,…,…
2447269468,466403924,1625586760566,2,4,14,0,0
2447271640,465956327,1625584771979,2,1,17,1,143
2447271640,465956327,1625584771979,2,1,17,0,0


In [ ]:
# 在is_clicked和duration上加和即可(因为一条点击数据对应一条相同的未点击数据，而异常值-1正好都是点击数据，不用担心出错)
train_data = train_data.group_by([
    "user_id", "article_id", "expose_time", 
    "network_env", "refresh_count", "expose_pos"
]).agg(
    pl.col("is_clicked").sum().alias("is_clicked"),  # is_clicked 列求和
    pl.col("duration").sum().alias("duration")       # duration 列求和
)
train_data

user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration
i64,i64,i64,i64,i64,i64,i64,i64
2382354600,462989776,1624654153051,5,35,142,0,0
2395959330,466618776,1625552823540,5,8,85,0,0
1417131344,465035343,1625140369203,5,1,9,0,0
1620297884,466379834,1625585195357,2,0,16,1,234
2445414622,465457598,1625235071269,2,6,72,0,0
…,…,…,…,…,…,…,…
2317527996,466094674,1625417188110,2,0,16,0,0
2228697026,463113764,1624691840608,5,6,28,0,0
1971991350,465499508,1625320896098,5,13,142,0,0


### 3. 拆分数据

#### 3.1 线下数据

In [ ]:
temp_path = "./temp_results" 

# 先添加 date 列
train_data = train_data.with_columns(
    pl.from_epoch(pl.col("expose_time") // 1000).cast(pl.Date).alias("date")
)

# 设定验证日期
val_date = pl.lit("2021-07-05").cast(pl.Date)
train_data_offline = train_data.filter(pl.col("date") < val_date)
val_data_offline =  train_data.filter(pl.col("date") == val_date)


# 保存数据
train_data_offline.write_ipc(f"{temp_path}/train_data_offline.ipc")
val_data_offline.write_ipc(f"{temp_path}/val_data_offline.ipc")


#### 3.2 线上数据

In [ ]:
temp_path = "/data1/zxh/news_rec/temp_results" 

# 设定验证日期
val_date = pl.lit("2021-07-05").cast(pl.Date)

train_data_online = train_data.filter(pl.col("date") <= val_date)
test_data_online = train_data.filter(pl.col("date") > val_date)


# 保存数据
train_data_online.write_ipc(f"{temp_path}/train_data_online.ipc")
test_data_online.write_ipc(f"{temp_path}/test_data_online.ipc")